In [1]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import time
from lxml import etree
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

ua = UserAgent(use_cache_server=False)
url_root = "https://www.youtube.com/results?search_query=interview&sp=EgIQAw%253D%253D"
url_head = "https://www.youtube.com"
header = {
        'Accept': 'text/html,*/*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Connection': 'keep-alive',
        'User-Agent': ua.random,
        'X-Requested-With': 'XMLHttpRequest'
    }
chrome_options = Options()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--disable-gpu')
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
page = [] ##初始化在这个位置！！！！


In [12]:
## 主页信息爬取

driver = webdriver.Chrome(r'C:\Program Files (x86)\Google\Chrome\chromedriver_win32\chromedriver')
driver.get(url_root)
for i in range(0,20):
    time.sleep(10)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    print("loading:%s" %i)
page.append(driver.page_source)

loading:0
loading:1
loading:2
loading:3
loading:4
loading:5
loading:6
loading:7
loading:8
loading:9
loading:10
loading:11
loading:12
loading:13
loading:14
loading:15
loading:16
loading:17
loading:18
loading:19


In [84]:
## 保存初始列表
filename = 'listspage.txt'
with open(filename, 'w', encoding = 'utf-8') as f1:
    f1.write(page[1])

In [15]:
web_lists = etree.HTML(page[1])
viewmore = web_lists.xpath('//yt-formatted-string[@id="view-more"]/a/@href')

viewlist = []
for line in viewmore:
    viewlist.append(url_head+str(line))

In [45]:
## 数据库初始化
import pandas
from pandas import DataFrame
detailpage = {"href":[]}
detailpage = DataFrame(detailpage)
youtube = {'uper':[],
           'href':[],
           'describe':[],
           'upload_time':[],
           'duration':[],
           'viewcount':[],
           'upscount':[],
           'downscount':[],
           'rel_uper':[],
           'rel_title':[],
           'rel_href':[],
           'rel_count':[]}
youtube = DataFrame(youtube)

In [2]:
## 数据库读入
import pandas as pd
detailpage = pd.read_csv("detailpage.csv")
youtube = pd.read_csv("youtube_new.csv")

In [3]:
print(len(youtube))

481


In [47]:
## 列表页信息爬取
driver = webdriver.Chrome(r'C:\Program Files (x86)\Google\Chrome\chromedriver_win32\chromedriver')
for i in range(0,len(viewlist)):
    status = True
    driver.get(viewlist[i])
    time.sleep(5)
    while status:
        try:
            driver.find_element(By.XPATH,'//div[@id="content"]/a')
            status = False
        except:
            driver.refresh()
            time.sleep(5)
    
    viewvideo = etree.HTML(driver.page_source)
    hreflist = viewvideo.xpath('//div[@id="content"]/a/@href')
    for line in hreflist:
        detailpage = detailpage.append([{"href":line}],ignore_index = True)
    if (i % 20 == 0):
        print("loading:%s" %i)

loading:0
loading:20
loading:40
loading:60
loading:80
loading:100


In [81]:
detailpage.to_csv("detailpage.csv",index = False, encoding = 'utf-8_sig')

In [4]:
##详情页初始化
import re
clean_re = re.compile('(<.*?>)|(&#.*?;)|( )')
buttons_re = re.compile('aria-label=(.*?)>')
buttons_clean_re = re.compile('(&#(.*?);)|( )')
reluper_re = re.compile(u'(来自|by)(.*?) ')
rel_count_re = re.compile(u' ([0-9]{1,3}(,([0-9]){3})*) (次观看|views)')
href_re = re.compile(u'href="(.*?)"')
title_re = re.compile(u'title="(.*?)"')

In [13]:
print(i)

482


In [ ]:
## 详情页爬取
driver = webdriver.Chrome(r'C:\Program Files (x86)\Google\Chrome\chromedriver_win32\chromedriver',options = chrome_options)
for i in range(482,len(detailpage['href'])):
    addr = url_head+detailpage['href'][i]
    detail = [] ###注意这里有一个初始化！！！
    driver.get(addr)
    status = True
    time.sleep(5)
    while status:
        try:
            driver.find_element(By.XPATH,'//yt-formatted-string[@id="owner-name"]/a')
            status = False
        except:
            driver.refresh()
            time.sleep(5)
    
    detail = driver.page_source
    web = etree.HTML(detail)

    uper = web.xpath('//yt-formatted-string[@id="owner-name"]/a/text()')
    desc = web.xpath('//div[@id="description"]/yt-formatted-string/text()')
    uptime = web.xpath('//div[@id="upload-info"]/span/text()')
    duration = web.xpath('//span[@class="ytp-time-duration"]/text()')
    viewcount = web.xpath('//div[@id="count"]/yt-view-count-renderer/span[1]/text()')

    buttons_raw = web.xpath('//div[@id="top-level-buttons"]/ytd-toggle-button-renderer/a/yt-icon-button/button[@id="button"]')
    ups_text_raw = etree.tostring(buttons_raw[0])
    downs_text_raw = etree.tostring(buttons_raw[1])
    ups_text = buttons_re.findall(str(ups_text_raw))
    downs_text = buttons_re.findall(str(downs_text_raw))
    ups = buttons_clean_re.sub("",str(ups_text))
    downs = buttons_clean_re.sub("",str(downs_text))
    
    relevant = web.xpath('//div[@class="metadata style-scope ytd-compact-video-renderer"]')
    relevant_href = web.xpath('//div[@class="metadata style-scope ytd-compact-video-renderer"]/a/@href')
    relevant_info_raw = web.xpath('//div[@class="metadata style-scope ytd-compact-video-renderer"]/a/h3/span')
    
    relevant_title = []
    relevant_uper = []
    relevant_count = []
    for info in relevant_info_raw:
        relevant_info_text = etree.tostring(info, encoding = 'Unicode',pretty_print = True, method = "html")
        relevant_title.append(title_re.findall(str(relevant_info_text)))
        relevant_uper.append(reluper_re.findall(str(relevant_info_text)))
        relevant_count.append(rel_count_re.findall(str(relevant_info_text))[0][0])
    
    youtube = youtube.append([{'uper':uper,
                               'href':addr,
                               'describe':desc,
                               'upload_time':uptime,
                               'duration':duration,
                               'viewcount':viewcount,
                               'upscount':ups,
                               'downscount':downs,
                               'rel_uper':relevant_uper,
                               'rel_title':relevant_title,
                               'rel_href':relevant_href,
                               'rel_count':relevant_count}],ignore_index = True)
    
    if (i%20 == 0):
        local = time.strftime('%H:%M:%S',time.localtime(time.time()))
        print("loading:%s：%s" %(i,local))
    if (i%100 == 0):
        youtube.to_csv("youtube_new.csv",index = False, encoding = 'utf-8_sig')

In [39]:
print(relevant_info_text)

<span id="video-title" class="style-scope ytd-compact-video-renderer" aria-label="David Letterman Mathematics Genius Prodigy Daniel Tammet Math 3.14 Pi Day by Jonathan J Crabtree 9 years ago 8 minutes, 14 seconds 8,566,407 views" title="David Letterman Mathematics Genius Prodigy Daniel Tammet Math 3.14 Pi Day">
              David Letterman Mathematics Genius Prodigy Daniel Tammet Math 3.14 Pi Day
            </span>
          



In [19]:
youtube.to_csv("youtube_new.csv",index = False, encoding = 'utf-8_sig')

In [241]:
## 详情页信息
for line in detail:
    web = etree.HTML(line)

    uper = web.xpath('//yt-formatted-string[@id="owner-name"]/a/text()')
    desc = web.xpath('//div[@id="description"]/yt-formatted-string/text()')
    uptime = web.xpath('//div[@id="upload-info"]/span/text()')
    duration = web.xpath('//span[@class="ytp-time-duration"]/text()')
    viewcount = web.xpath('//div[@id="count"]/yt-view-count-renderer/span[1]/text()')

    buttons_raw = web.xpath('//div[@id="top-level-buttons"]/ytd-toggle-button-renderer/a/yt-icon-button/button[@id="button"]')
    ups_text_raw = etree.tostring(buttons_raw[0])
    downs_text_raw = etree.tostring(buttons_raw[1])
    ups_text = buttons_re.findall(str(ups_text_raw))
    downs_text = buttons_re.findall(str(downs_text_raw))
    ups = buttons_clean_re.sub("",str(ups_text))
    downs = buttons_clean_re.sub("",str(downs_text))
    
    relevant = web.xpath('//div[@class="metadata style-scope ytd-compact-video-renderer"]')
    relevant_href = web.xpath('//div[@class="metadata style-scope ytd-compact-video-renderer"]/a/@href')
    relevant_info_raw = web.xpath('//div[@class="metadata style-scope ytd-compact-video-renderer"]/a/h3/span')
    
    relevant_title = []
    relevant_uper = []
    relevant_count = []
    for info in relevant_info_raw:
        relevant_info_text = etree.tostring(info, encoding = 'Unicode',pretty_print = True, method = "html")
        relevant_title.append(title_re.findall(str(relevant_info_text)))
        relevant_uper.append(reluper_re.findall(str(relevant_info_text)))
        relevant_count.append(rel_count_re.findall(str(relevant_info_text))[0][0])
    
    youtube = youtube.append([{'uper':uper,
           'describe':desc,
           'upload_time':uptime,
           'duration':duration,
           'viewcount':viewcount,
           'upscount':ups,
           'downscount':downs,
           'rel_uper':relevant_uper,
           'rel_title':relevant_title,
           'rel_href':relevant_href,
           'rel_count':relevant_count}],ignore_index = True)

In [48]:
## 主页信息分析 - 8月21日mode
import re
web = etree.HTML(page[0])
block_raw  = web.xpath('//a[@id = "video-title"]')

block_text = []
href = []
title = []
for line in block_raw:
    texts = etree.tostring(line, encoding = 'Unicode')
    href.append(href_re.findall(texts))
    title.append(title_re.findall(texts))

weblist =[]
for line in href:
    weblist.append(url_head+line[0])

In [235]:
# 监测输出窗口
for line in detail:
    print(len(line))

539495
435336
434203
435421
430630
449654
533361
439554
435127
430559
433368
414192
428081
419836


In [242]:
youtube.to_csv("youtube_new.csv",index = False, encoding = 'utf-8_sig')